In [3]:
#load data from csv
import csv
input_file_path = "/Users/k.abhishek/Documents/customer_clusters/export_all_v2.csv"
def load_data(input_file_path):
    with open(input_file_path, mode ='r') as file:
        csvFile = csv.reader(file)
        data = list(csvFile)
    return data

data = load_data(input_file_path)

In [4]:
countries_in_focus = ["IN", "CN", "US"]
# countries_in_focus = ["IN"]
country_filter = set(countries_in_focus)

#filtering based on country code
# filtered_data = [ d for d in data if d[15] in country_filter]

filtered_data = data.copy()
import random 
random.seed(42)
random.shuffle(filtered_data)

In [5]:
len(filtered_data)

3610594

In [35]:
#find the unique values of legal_cluster_id

legal_cluster_id_set = set([d[1] for d in filtered_data])
print(len(legal_cluster_id_set))

legal_cluster_id_to_data = {}
count_null_legal_cluster_id = 0
for d in filtered_data:
    if d[1] == "null":
        count_null_legal_cluster_id +=1
        continue
    if legal_cluster_id_to_data.get(d[1]) is None:
        legal_cluster_id_to_data[d[1]] = [d]
    else:
        legal_cluster_id_to_data[d[1]].append(d)    
# filtered_data = []
num_entries = 100000
test_corpus = []
comparison_corpus = []
count_entries = 0
for legal_cluster_id, entry in legal_cluster_id_to_data.items():
    # if count_entries > num_entries:
    #     break
    if len(entry) > 1:
        count_entries += len(entry)
        test_corpus.append(entry[0])
        comparison_corpus.extend(entry[1:])
    else:
        comparison_corpus.extend(entry)


2790777


In [36]:
country_wise_corpus = {}
for entry in comparison_corpus:
    country_code = entry[15]
    if country_wise_corpus.get(country_code) is None:
        country_wise_corpus[country_code] = [entry]
    else:
        country_wise_corpus[country_code].append(entry)

In [37]:
import numpy as np
entries_per_legal_cluster_id = [len(v) for k,v in legal_cluster_id_to_data.items()]
# entries_per_legal_cluster_id = [len(v) for k,v in legal_cluster_id_to_data.items() if len(v) > 1]
mean = np.mean(entries_per_legal_cluster_id)
std = np.std(entries_per_legal_cluster_id)
print(mean, std)

1.2266229894480962 1.0011390672114102


In [38]:
sum(entries_per_legal_cluster_id)

3423230

mean and standard deviation for the size of legal_cluster_id
1.5947940644465912 2.2155833789806207

mean and standard deviation for the size of legal_cluster_id after filtering the size of cluster to be > 1
5.20206364596017 4.4183111370491055

In [39]:
count_null_legal_cluster_id

187364

In [40]:
print(len(filtered_data))
print(len(test_corpus))
print(len(comparison_corpus))

3610594
395029
3028201


In [41]:
def extract_tax_numbers(arr):
    unique_tax_numbers = set()
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            unique_tax_numbers.add(arr[i].lower())
    return unique_tax_numbers

def extract_bvd_numbers(arr):
    unique_bvd_numbers = set()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            unique_bvd_numbers.add(arr[i].lower())
    return unique_bvd_numbers

def match_tax(arr, unique_tax_numbers):
    tax = extract_tax_numbers(arr)
    for tax_num in tax:
        if tax_num in unique_tax_numbers:
            return unique_tax_numbers[tax_num]
    return None

def match_bvd(arr, unique_bvd_numbers):
    bvd = extract_bvd_numbers(arr)
    for bvd_num in bvd:
        if bvd_num in unique_bvd_numbers:
            return unique_bvd_numbers[bvd_num]
    return None

In [42]:
unique_tax_numbers = {}
unique_bvd_numbers = {}
for idx, arr in enumerate(comparison_corpus):
    country_code = arr[15]
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            if unique_bvd_numbers.get(arr[i].lower()):
                unique_bvd_numbers[arr[i].lower()].append(arr)
            else:
                unique_bvd_numbers[arr[i].lower()] = [arr]
            # unique_bvd_numbers[arr[i]] = arr
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            if unique_tax_numbers.get(arr[i].lower()):
                unique_tax_numbers[arr[i].lower()].append(arr)
            else:
                unique_tax_numbers[arr[i].lower()] = [arr]
            # unique_tax_numbers[arr[i]] = arr


In [43]:
import re
import numpy as np
from rapidfuzz import fuzz
from annoy import AnnoyIndex

import re
stop_words = {'llp', 'ltd', 'llc', 'plc', 'inc', 'corp', 'nv', 'group', 'gmbh', 'company', 'corporation', 'limited'}
delimiters = {".", ",", "-", "_", "'", "&", "/", "\\", ":", ";", "(", ")", "[", "]", "{", "}", " "}
def split_company_name(name):
    pattern = '|'.join(map(re.escape, delimiters))
    words = re.split(pattern, name)
    return [word for word in words if word]

def lowercase_company_name(name):
    return name.lower()

# List of common delimiters
def remove_stop_words(words):
    # return " ".join(words)
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

def preprocess_name_2(trading_name):
    # Lowercase the company name
    trading_name = lowercase_company_name(trading_name)
    # Split the company name into words
    words = split_company_name(trading_name)

    # Remove stop words
    clean_name = remove_stop_words(words)
    return clean_name

# Preprocessing function
def preprocess_name_1(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    tokens = name.split()
    tokens.sort()  # Sort tokens for token_sort_ratio
    return ' '.join(tokens)

preprocess_name = preprocess_name_2

In [62]:

def name_to_vector(name, embedding_dim=128):
    vector = np.zeros(embedding_dim)
    tokens = name.split()
    for token in tokens:
        for char in token:
            vector[ord(char) % embedding_dim] += 1  
    return vector

def build_index(corpus, embedding_dim=128, metric='angular', num_trees=25):
    # Preprocess the entire corpus
    # corpus_of_names = ["John Doe", "Jane Smith", "Alice Johnson"]  # Add all million names here
    corpus_of_names = [d[4] for d in corpus]
    preprocessed_names = [preprocess_name(name) for name in corpus_of_names]

    annoy_index = AnnoyIndex(embedding_dim, metric)
    # Convert names to vectors (simple bag-of-words vectorization)


    name_vectors = [name_to_vector(name, embedding_dim) for name in preprocessed_names]

    for i, vector in enumerate(name_vectors):
        # print(i, vector)
        annoy_index.add_item(i, vector)

    annoy_index.build(num_trees) # 50 trees
    return annoy_index, corpus_of_names, preprocessed_names


In [63]:
# # Preprocess the entire corpus
# # corpus_of_names = ["John Doe", "Jane Smith", "Alice Johnson"]  # Add all million names here
# corpus_of_names = [d[4] for d in comparison_corpus]
# preprocessed_names = [preprocess_name(name) for name in corpus_of_names]

# # Build Annoy index
# embedding_dim = 128  # Dimensionality of the embeddings
# annoy_index = AnnoyIndex(embedding_dim, 'angular')

# # Convert names to vectors (simple bag-of-words vectorization)
# def name_to_vector(name):
#     vector = np.zeros(embedding_dim)
#     tokens = name.split()
#     for token in tokens:
#         for char in token:
#             vector[ord(char) % embedding_dim] += 1  
#     return vector

# name_vectors = [name_to_vector(name) for name in preprocessed_names]

# for i, vector in enumerate(name_vectors):
#     # print(i, vector)
#     annoy_index.add_item(i, vector)

# annoy_index.build(50) # 50 trees

In [64]:
# Fuzzy matching score function
function_selection = {
    "ratio": 1,
    "partial_ratio": 1,
    "token_sort_ratio": 1,
    "token_set_ratio": 1,
    "partial_token_sort_ratio": 1,
    "partial_token_set_ratio": 1
}
def combined_fuzzy_score(name1, name2):
    ratio = fuzz.ratio(name1, name2) * function_selection["ratio"]
    partial_ratio = fuzz.partial_ratio(name1, name2) * function_selection["partial_ratio"]
    token_sort_ratio = fuzz.token_sort_ratio(name1, name2)  * function_selection["token_sort_ratio"]
    token_set_ratio = fuzz.token_set_ratio(name1, name2)    * function_selection["token_set_ratio"]
    partial_token_sort_ratio = fuzz.partial_token_sort_ratio(name1, name2) * function_selection["partial_token_sort_ratio"]
    partial_token_set_ratio = fuzz.partial_token_set_ratio(name1, name2) * function_selection["partial_token_set_ratio"]
    # print(f"ratio: {ratio}, partial_ratio: {partial_ratio}, token_sort_ratio: {token_sort_ratio}, token_set_ratio: {token_set_ratio}, partial_token_sort_ratio: {partial_token_sort_ratio}, partial_token_set_ratio: {partial_token_set_ratio}")
    # print(f"Ratios: {ratio}, {partial_ratio}, {token_sort_ratio}, {token_set_ratio}, {partial_token_sort_ratio}, {partial_token_set_ratio}")
    return sum([ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio])/sum(function_selection.values())
    # return max(ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio)

In [65]:
embedding_dim = 128
num_trees = 25
index_name_vector_clustered_by_country = {}
preprocessed_names_by_country = {}
corpus_of_names_by_country = {}
for country_code, corpus in country_wise_corpus.items():
    index_name_vector_clustered_by_country[country_code], corpus_of_names_by_country[country_code], preprocessed_names_by_country[country_code] = build_index(corpus, num_trees=num_trees)

In [66]:
# Function to find best match
def find_best_match(input_name, country_code, annoy_index, embedding_dim = 128, num_candidates=15):
    preprocessed_input = preprocess_name(input_name)
    input_vector = name_to_vector(preprocessed_input, embedding_dim)
    # Retrieve nearest neighbors using Annoy
    nearest_neighbors = annoy_index.get_nns_by_vector(input_vector, num_candidates, include_distances=False)
    # print(nearest_neighbors)
    best_match_score = 0
    best_match_name = None
    best_match_idx = None
    preprocessed_names = preprocessed_names_by_country[country_code]
    corpus_of_names = corpus_of_names_by_country[country_code]
    for neighbor_idx in nearest_neighbors:
        candidate = preprocessed_names[neighbor_idx]
        score = combined_fuzzy_score(preprocessed_input, candidate)
        if score > best_match_score:
            best_match_score = score
            best_match_name = corpus_of_names[neighbor_idx]
            best_match_idx = neighbor_idx
    
    return best_match_name, best_match_score, best_match_idx


In [67]:
from tqdm import tqdm
success = 0
failure = 0
not_applicable = 0
result = {}
false_positives = {}
qualified_as_match_threshold = 0
correctly_qualified_as_match_threshold = 0
count_entries_name_matching =0
threshold = 90
for arr in tqdm(test_corpus):
    legal_cluster_id = arr[1].lower()
    matches = {}
    #tax matching
    tax_match = match_tax(arr, unique_tax_numbers)
    if tax_match:
        success += 1
        matches[1] = tax_match
        result[str(arr)] = matches
        continue
    #bvd matching
    bvd_match = match_bvd(arr, unique_bvd_numbers)
    if bvd_match:
        success += 1
        matches[1] = bvd_match
        result[str(arr)] = matches
        continue
    # print("Name matching")
    company_name = arr[4]
    count_entries_name_matching += 1
    country_code = arr[15]
    annoy_index = index_name_vector_clustered_by_country[country_code]
    corpus = country_wise_corpus[country_code]
    best_match_name, best_match_score, best_match_idx = find_best_match(company_name, country_code, annoy_index, embedding_dim=128, num_candidates=15)
    best_matched_entry = None
    if best_match_idx == None:
        not_applicable +=1
        print(arr)
    if best_match_idx and best_match_score > threshold:
        # best_matched_entry = comparison_corpus[best_match_idx]
        best_matched_entry = corpus[best_match_idx]
        # print(best_match_score)
        qualified_as_match_threshold +=1
        if legal_cluster_id == best_matched_entry[1]:
            success +=1
            correctly_qualified_as_match_threshold +=1
        else:
            false_positives[str(arr)] = best_matched_entry
            failure +=1
    else:
        failure +=1
    result[str(arr)] = best_matched_entry
    # print(idx+1, success, failure, success + failure)


 16%|█▋        | 64627/395029 [00:30<02:17, 2394.50it/s]

['c330c792-29bf-3c51-9931-542003abcc41', 'f41ba3d6-8435-3dcb-b5e5-01454c30a1bb', 'RU00186779', 'Customer', 'LIMITED COMPANY', '9', 'STRELNIKOVA STR 9,', 'null', 'null', 'null', 'null', 'VLADIVOSTOK', 'null', 'null', 'null', 'RU', 'Russia', 'TEL', 'KZ', '7', 'null', '9147077761', '2016-06-05T17:18:57.525Z', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']


 89%|████████▉ | 352986/395029 [02:35<00:18, 2303.08it/s]

['6b266ac5-f9b4-3008-833c-4cda137a33b8', 'd804d709-19e9-3231-adb4-9ef98fcc7f64', 'QA50002951', 'Customer', 'GROUP COMPANY', 'null', 'GATE NO. A2, 1ST FLOOR', '7303', 'RETAJ BUILDING, SALWA ROAD', 'null', 'null', 'DOHA', 'null', 'null', 'null', 'QA', 'Qatar', 'TEL', 'QA', '974', 'null', '44681144', '2020-06-20T08:27:05.955Z', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']


100%|██████████| 395029/395029 [02:55<00:00, 2255.48it/s]


In [69]:
print(success, failure)
print(qualified_as_match_threshold, correctly_qualified_as_match_threshold)

322201 72828
206515 185516


311033 83997
205553 177854
0.7873655165430473


In [70]:
success/(success + failure)

0.8156388518311314

In [71]:
count_entries_name_matching

258344

In [72]:
precision = correctly_qualified_as_match_threshold/qualified_as_match_threshold
print(precision)
qualified_as_match_threshold - correctly_qualified_as_match_threshold

0.8983173135123357


20999

In [61]:
import os
save_result_filename = "complete_data_" + "false_positives_" + str(embedding_dim) + "_" + str(function_selection['ratio']) + "_" + str(function_selection['partial_ratio']) + "_" + str(function_selection['token_sort_ratio']) + "_" + str(function_selection['token_set_ratio']) + "_" + str(function_selection['partial_token_sort_ratio']) + "_" + str(function_selection['partial_token_set_ratio']) + "_threshold_" + str(threshold) + ".json"
# save_result_filename = "result_name_matching_threshold_85.json"
save_folder_path = "/Users/k.abhishek/Documents/customer_clusters/experiments/approximate_match_approach/results"
save_result_file_path = os.path.join(save_folder_path, save_result_filename)

def save_result(result, save_result_file_path):
    import json
    with open(save_result_file_path, 'w') as f:
        json.dump(result, f)

save_result(false_positives, save_result_file_path)